In [23]:
import uproot
import awkward as ak
import numpy as np
import vector
vector.register_numba()
vector.register_awkward()


In [18]:
columns = [
    "era",
    "higgs1_reco_pt",
    "higgs1_reco_eta",
    "higgs1_reco_phi",
    "higgs1_reco_mass",
    "higgs2_reco_pt",
    "higgs2_reco_eta",
    "higgs2_reco_phi",
    "higgs2_reco_mass",
    "HT",
    "higgs1_DeltaRjj",
    "higgs2_DeltaRjj",
    "minDeltaR_Higgjj",
    "maxDeltaR_Higgjj",
    "higgs1_helicityCosTheta",
    "higgs2_helicityCosTheta",
    "hh_CosThetaStar_CS",
    "hh_vec_mass",
    "hh_vec_pt",
    "hh_vec_eta",
    "hh_vec_DeltaR",
    "hh_vec_DeltaPhi",
    "hh_vec_DeltaEta",
    "higgs1_reco_jet1_pt",
    "higgs1_reco_jet1_eta",
    "higgs1_reco_jet1_phi",
    "higgs1_reco_jet1_mass",
    "higgs1_reco_jet2_pt",
    "higgs1_reco_jet2_eta",
    "higgs1_reco_jet2_phi",
    "higgs1_reco_jet2_mass",
    "higgs2_reco_jet1_pt",
    "higgs2_reco_jet1_eta",
    "higgs2_reco_jet1_phi",
    "higgs2_reco_jet1_mass",
    "higgs2_reco_jet2_pt",
    "higgs2_reco_jet2_eta",
    "higgs2_reco_jet2_phi",
    "higgs2_reco_jet2_mass",
    "add_jet1pt_pt",
    "add_jet1pt_eta",
    "add_jet1pt_phi",
    "add_jet1pt_mass",
    "sigma_over_higgs1_reco_mass",
    "sigma_over_higgs2_reco_mass",
]
main_dir='/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/keras_models_morphing'


In [19]:
input_root=f'/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/JetMET_2022EE_2b_signal_region_to_4b_soumya_january2025.root'
tree=uproot.open(input_root)["tree"]
print(tree.keys())


['era', 'event', 'HT', 'jet_pt', 'jet_pt_raw', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_mass_raw', 'jet_deepjet_probBVsAll', 'jet_pnet_probBVsAll', 'jet_pnet_probTauVsJet', 'jet_pnet_probb', 'jet_pnet_probc', 'jet_pnet_probuds', 'jet_pnet_probtauh', 'jet_pnet_ptcorr', 'jet_pnet_ptnu', 'jet_pnet_ptres', 'jet_pnet_jec_L1L2L3', 'jet_pnet_jec_L2Rel', 'jet_pnet_jec_L2L3Res', 'btag_cat', 'weight', 'is_boosted', 'weight_dnn', 'sigma_higgs1', 'sigma_over_higgs1_reco_mass', 'higgs1_reco_pt', 'higgs1_reco_eta', 'higgs1_reco_phi', 'higgs1_reco_mass', 'sigma_higgs2', 'sigma_over_higgs2_reco_mass', 'higgs2_reco_pt', 'higgs2_reco_eta', 'higgs2_reco_phi', 'higgs2_reco_mass', 'hh_vec_mass', 'hh_vec_pt', 'hh_vec_eta', 'hh_vec_phi', 'hh_vec_DeltaR', 'hh_vec_DeltaPhi', 'hh_vec_DeltaEta', 'hh_vec_ptOmass', 'njet', 'higgs1_reco_jet1_pt', 'higgs1_reco_jet1_eta', 'higgs1_reco_jet1_phi', 'higgs1_reco_jet1_mass', 'higgs1_reco_jet2_pt', 'higgs1_reco_jet2_eta', 'higgs1_reco_jet2_phi', 'higgs1_reco_jet2_mass', 'hig

get the columns from the tree

In [20]:
events = tree.arrays(columns, library="ak")
n_events = 10
print(events[:n_events] , type(events.era), len(events['era']))


[{era: 1, higgs1_reco_pt: 115, ... sigma_over_higgs2_reco_mass: 0.129}] <class 'awkward.highlevel.Array'> 2180220


In [87]:
# given his snippet of code in c++ translate in python using akward arrays and vectors

# float HelicityCosTheta(TLorentzVector Booster, TLorentzVector Boosted)
# {
#     TVector3 BoostVector = Booster.BoostVector();
#     Boosted.Boost( -BoostVector.x(), -BoostVector.y(), -BoostVector.z() );
#     return Boosted.CosTheta();
# }

# higgs1_helicityCosTheta =fabs(HelicityCosTheta( leadingHiggsCands.at(best_pairing_index)  ,  leading_higgs_leading_jet ));
# higgs2_helicityCosTheta =fabs(HelicityCosTheta( subleadingHiggsCands.at(best_pairing_index), subleading_higgs_leading_jet ));

# //Costhetastar in CS frame
# cosine of the polar angle of one Higgs boson with respect to the beam axis in the HH rest frame
# TLorentzVector higgs1_vec;
# higgs1_vec =  leadingHiggsCands.at(best_pairing_index);
# higgs1_vec.Boost( - hh_vec.BoostVector());
# hh_CosThetaStar_CS = fabs(higgs1_vec.CosTheta());


def HelicityCosTheta(higgs, jet):
    # print("jet px",jet.px)
    # print("higgs px",higgs.px)
    # print("jet theta",jet.theta)
    higgs_velocity = higgs.to_beta3()
    jet = jet.boost_beta3(-higgs_velocity)
    # print("jet px",jet.px)
    # print("jet theta",jet.theta)
    return np.cos(jet.theta)


def Costhetastar_CS(higgs1_vec, hh_vec):
    # print("px",higgs1_vec.px)
    # print("theta",higgs1_vec.theta)
    hh_velocity = hh_vec.to_beta3()
    higgs1_vec = higgs1_vec.boost_beta3(-hh_velocity)
    # print("px",higgs1_vec.px)
    # print("theta",higgs1_vec.theta)
    return abs(np.cos(higgs1_vec.theta))


higgs1_vec = ak.zip(
    {
        "pt": events.higgs1_reco_pt,
        "eta": events.higgs1_reco_eta,
        "phi": events.higgs1_reco_phi,
        "mass": events.higgs1_reco_mass,
    },
    with_name="Momentum4D",
)
higgs2_vec = ak.zip(
    {
        "pt": events.higgs2_reco_pt,
        "eta": events.higgs2_reco_eta,
        "phi": events.higgs2_reco_phi,
        "mass": events.higgs2_reco_mass,
    },
    with_name="Momentum4D",
)
print(higgs1_vec, type(higgs1_vec))
print(higgs1_vec.px)
print(higgs2_vec.px)
hh_vec = higgs1_vec + higgs2_vec
print(hh_vec.px, '\n\n')

higgs1_jet1_vec = ak.zip(
    {
        "pt": events.higgs1_reco_jet1_pt,
        "eta": events.higgs1_reco_jet1_eta,
        "phi": events.higgs1_reco_jet1_phi,
        "mass": events.higgs1_reco_jet1_mass,
    },
    with_name="Momentum4D",
)
higgs2_jet1_vec = ak.zip(
    {
        "pt": events.higgs2_reco_jet1_pt,
        "eta": events.higgs2_reco_jet1_eta,
        "phi": events.higgs2_reco_jet1_phi,
        "mass": events.higgs2_reco_jet1_mass,
    },
    with_name="Momentum4D",
)

higgs1_helicityCosTheta = abs(HelicityCosTheta(higgs1_vec, higgs1_jet1_vec))
higgs2_helicityCosTheta = abs(HelicityCosTheta(higgs2_vec, higgs2_jet1_vec))
hh_CosThetaStar_CS = abs(Costhetastar_CS(higgs1_vec, hh_vec))
print(higgs1_helicityCosTheta)
print(higgs2_helicityCosTheta)
print(hh_CosThetaStar_CS)
higgs1_helicityCosTheta = abs(HelicityCosTheta(-higgs1_vec, higgs1_jet1_vec))
higgs2_helicityCosTheta = abs(HelicityCosTheta(-higgs2_vec, higgs2_jet1_vec))
hh_CosThetaStar_CS = abs(Costhetastar_CS(higgs1_vec, -hh_vec))
print(higgs1_helicityCosTheta)
print(higgs2_helicityCosTheta)
print(hh_CosThetaStar_CS)

[{pt: 115, eta: -0.585, phi: 0.511, mass: 128}, ... phi: 1.72, mass: 119}] <class 'vector.backends.awkward.MomentumArray4D'>
[99.9, 85.4, -166, 125, -123, 48.3, 132, ... 187, 276, -106, -169, 130, -147, -30.8]
[19.8, 15.2, 105, -94.5, 81.2, -72.5, -58.4, ... -121, 11.9, 73.9, -79.7, 122, 2.8]


[120, 101, -60.1, 30.8, -42, -24.2, 73.5, ... 155, -93.6, -94.7, 50.8, -24.9, -28] 


[0.117, 0.189, 0.714, 0.885, 0.468, 0.306, ... 0.881, 0.91, 0.876, 0.237, 0.785]
[0.0587, 0.961, 0.9, 0.614, 0.998, 0.0118, ... 0.62, 0.0157, 0.288, 0.975, 0.689]
[0.405, 0.57, 0.737, 0.457, 0.888, 0.772, ... 0.46, 0.709, 0.16, 0.701, 0.521, 0.945]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [51]:
print(events.higgs1_helicityCosTheta)
print(events.higgs2_helicityCosTheta)
print(events.hh_CosThetaStar_CS)

[0.117, 0.189, 0.714, 0.885, 0.468, 0.306, ... 0.881, 0.91, 0.876, 0.237, 0.785]
[0.0587, 0.961, 0.9, 0.614, 0.998, 0.0118, ... 0.62, 0.0157, 0.288, 0.975, 0.689]
[0.405, 0.57, 0.737, 0.457, 0.888, 0.772, ... 0.46, 0.709, 0.16, 0.701, 0.521, 0.945]


In [52]:
boosted = higgs1_vec.boost(higgs1_vec.to_beta3())

print(higgs1_vec.to_beta3()[0])
print(boosted[0])
print(higgs1_vec[0])
print(type(higgs1_vec))
print(higgs1_vec.E[0])
print(np.sqrt(higgs1_vec.px[0]**2+higgs1_vec.py[0]**2))


{rho: 0.617, phi: 0.511, eta: -0.585, pt: 115, mass: 128}
{x: 290, y: 163, z: -206, tau: 128, pt: 115, mass: 128}
{pt: 115, eta: -0.585, phi: 0.511, mass: 128}
<class 'vector.backends.awkward.MomentumArray4D'>
185.758544921875
114.53314961865229


In [53]:
v = ak.zip(
    {
        "pt": 1,
        "eta": 0,
        "phi": 0,
        "mass": 1,
    },
    with_name="Momentum4D",
)
print(v.px)
print(v.py)
print(v.pz)
print(v.gamma)
v1=v.boost(-v)
print(v1.px)
print(v1.py)
print(v1.y)
print(v1.pz)
print(v1.z)
print(v1)

1.0
0.0
0.0
1.4142135623730951
6.661338147750939e-16
-1.2246467991473522e-16
-1.2246467991473522e-16
0.0
0.0
{x: 6.66e-16, y: -1.22e-16, z: 0, tau: 1, pt: 1, mass: 1}


In [54]:
v1=v.boost(v)
print(v1.px)
print(v1.py)
print(v1.y)
print(v1.pz)
print(v1.z)
print(v1.gamma)
print(v1)

2.8284271247461903
0.0
0.0
0.0
0.0
3.0000000000000004
{x: 2.83, y: 0, z: 0, tau: 1, pt: 1, mass: 1}


In [61]:
p1=ak.zip(
    {
        "E": 10,
        "px": 3,
        "py": 4,
        "pz": 0,
    },
    with_name="Momentum4D",
)
p2=ak.zip(
    {
        "E": 8,
        "px": -2,
        "py": -1,
        "pz": 5,
    },
    with_name="Momentum4D",
)
print(p1.px)
print(p1.py)
print(p1.pt)
print(p1.mass,np.sqrt(75))
p2_boosted=p2.boost(-p1)
print(p2_boosted)

3
4
5.0
8.660254037844387 8.660254037844387
{x: -2.19, y: -1.25, z: 5, t: -8.08, E: 8, px: -2, py: -1, pz: 5}


In [68]:
higgs2_vec = ak.zip(
    {
        "pt": events.higgs2_reco_pt,
        "eta": events.higgs2_reco_eta,
        "phi": events.higgs2_reco_phi,
        "mass": events.higgs2_reco_mass,
    },
    with_name="Momentum4D",
)
higgs2_vec_alt = ak.zip(
    {
        "px": events.higgs2_reco_pt*np.cos(events.higgs2_reco_phi),
        "py": events.higgs2_reco_pt*np.sin(events.higgs2_reco_phi),
        "pz": events.higgs2_reco_pt*np.sinh(events.higgs2_reco_eta),
        "E": np.sqrt(events.higgs2_reco_pt**2+(events.higgs2_reco_pt*np.sinh(events.higgs2_reco_eta))**2+events.higgs2_reco_mass**2),
    },
    with_name="Momentum4D",
)

print(higgs2_vec.px)

print(higgs2_vec_alt.px)
print(higgs2_vec.py)
print(higgs2_vec_alt.py)
print(higgs2_vec.E)
print(higgs2_vec_alt.E)


print(-np.log(np.tan(higgs2_vec.theta/2)))
print(events.higgs2_reco_eta)


[19.8, 15.2, 105, -94.5, 81.2, -72.5, -58.4, ... -121, 11.9, 73.9, -79.7, 122, 2.8]
[19.8, 15.2, 105, -94.5, 81.2, -72.5, -58.4, ... -121, 11.9, 73.9, -79.7, 122, 2.8]
[95.3, -104, -11, 212, -42.5, 79.3, 3.85, ... 50.2, 62.1, -99.8, 233, -146, -59.1]
[95.3, -104, -11, 212, -42.5, 79.3, 3.85, ... 50.2, 62.1, -99.8, 233, -146, -59.1]
[190, 241, 187, 332, 176, 158, 143, 132, ... 539, 248, 387, 168, 275, 279, 403, 172]
[190, 241, 187, 332, 176, 158, 143, 132, ... 539, 248, 387, 168, 275, 279, 403, 172]
[-0.968, 1.36, -0.534, -0.823, -1.03, -0.0784, ... 0.787, 1.31, -0.137, -1.34, 1.51]
[-0.968, 1.36, -0.534, -0.823, -1.03, -0.0784, ... 0.787, 1.31, -0.137, -1.34, 1.51]


In [74]:
opp_higgs2_vec=-higgs2_vec
print(higgs2_vec)
print(opp_higgs2_vec)

print(higgs2_vec.px)
print(opp_higgs2_vec.px)
print(higgs2_vec.py)
print(opp_higgs2_vec.py)
print(higgs2_vec.pz)
print(opp_higgs2_vec.pz)
print(higgs2_vec.E)
print(opp_higgs2_vec.E)
print(higgs2_vec.theta)
print(opp_higgs2_vec.theta)
print(higgs2_vec.phi)
print(opp_higgs2_vec.phi)
print(higgs2_vec.mass)
print(opp_higgs2_vec.mass)


[{pt: 97.3, eta: -0.968, phi: 1.37, mass: 121}, ... phi: -1.52, mass: 98.9}]
[{rho: 97.3, phi: -1.78, eta: 0.968, tau: -121, ... pt: 59.2, mass: 98.9}]
[19.8, 15.2, 105, -94.5, 81.2, -72.5, -58.4, ... -121, 11.9, 73.9, -79.7, 122, 2.8]
[-19.8, -15.2, -105, 94.5, -81.2, 72.5, ... 121, -11.9, -73.9, 79.7, -122, -2.8]
[95.3, -104, -11, 212, -42.5, 79.3, 3.85, ... 50.2, 62.1, -99.8, 233, -146, -59.1]
[-95.3, 104, 11, -212, 42.5, -79.3, -3.85, ... -50.2, -62.1, 99.8, -233, 146, 59.1]
[-110, 192, -59.3, -214, -112, -8.43, -72.2, ... 343, 55, 214, -33.9, -340, 127]
[110, -192, 59.3, 214, 112, 8.43, 72.2, ... 194, -343, -55, -214, 33.9, 340, -127]
[190, 241, 187, 332, 176, 158, 143, 132, ... 539, 248, 387, 168, 275, 279, 403, 172]
[83.2, 193, 0, 298, 105, 0, 0, 0, 496, ... 0, 514, 205, 346, 0, 216, 214, 375, 99.8]
[2.42, 0.501, 2.08, 2.31, 2.46, 1.65, ... 0.364, 0.854, 0.526, 1.71, 2.63, 0.434]
[0.726, 2.64, 1.06, 0.827, 0.686, 1.49, ... 2.78, 2.29, 2.62, 1.43, 0.512, 2.71]
[1.37, -1.43, -0.10

AttributeError: no field named 'v'

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/awkward/highlevel.py#L1124)

In [86]:
velocity=higgs2_vec.to_beta3()
print(velocity[0])
print(velocity.x)
print(velocity.px)

velocity=(-higgs2_vec).to_beta3()
print(velocity[0])
print(velocity.x)
print(velocity.px)


print(higgs2_vec.x)
print(higgs2_vec.px)
print(higgs2_vec.gamma)

vx=higgs2_vec.px/higgs2_vec.E
print(vx)

{rho: 0.512, phi: 1.37, eta: -0.968, pt: 97.3, mass: 121}
[0.104, 0.0629, 0.563, -0.284, 0.462, ... 0.0709, 0.268, -0.286, 0.303, 0.0163]
[0.104, 0.0629, 0.563, -0.284, 0.462, ... 0.0709, 0.268, -0.286, 0.303, 0.0163]
{rho: 1.17, phi: -1.78, eta: 0.968, pt: 97.3, mass: 121}
[-0.238, -0.0784, -inf, 0.317, -0.772, inf, ... -inf, -0.342, 0.372, -0.326, -0.028]
[-0.238, -0.0784, -inf, 0.317, -0.772, inf, ... -inf, -0.342, 0.372, -0.326, -0.028]
[19.8, 15.2, 105, -94.5, 81.2, -72.5, -58.4, ... -121, 11.9, 73.9, -79.7, 122, 2.8]
[19.8, 15.2, 105, -94.5, 81.2, -72.5, -58.4, ... -121, 11.9, 73.9, -79.7, 122, 2.8]
[1.57, 2.37, 1.31, 3.19, 1.77, 1.36, 1.32, ... 3.17, 1.15, 2.28, 2.21, 3.86, 1.74]
[0.104, 0.0629, 0.563, -0.284, 0.462, ... 0.0709, 0.268, -0.286, 0.303, 0.0163]
